In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install kaggle -Uqq
!mkdir /root/.kaggle
!mkdir model_checkpoints
!cp /content/drive/MyDrive/kaggle.json  /root/.kaggle

In [ ]:
!pip install wandb -Uqq

In [ ]:
!kaggle datasets download -d preetviradiya/brian-tumor-dataset
!mkdir data
!unzip -d data -q /content/brian-tumor-dataset.zip
!rm -rf /content/brian-tumor-dataset.zip

In [ ]:
from collections import Counter
from pathlib import Path

import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

tf.keras.utils.set_random_seed(42)
tf.config.experimental.enable_op_determinism()

from tensorflow.keras import layers

import warnings

warnings.filterwarnings("ignore")

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import wandb
from wandb.integration.keras import WandbCallback
from pathlib import Path

In [ ]:
from dataclasses import dataclass


@dataclass
class Parameters:
    learning_rate: float = 0.001
    batch_size: int = 32
    data_dir = Path('/content/data/Brain Tumor Data Set/')
    train_dir = data_dir / 'Brain Tumor Data Set'
    verbose = 1
    epochs = 100
    img_size = 224
    img_shape = (img_size, img_size, 3)
    class_names = None
    seed = 42

config = Parameters()

In [ ]:
train_loader, val_loader = tf.keras.utils.image_dataset_from_directory(
  config.train_dir,
  validation_split=0.2,
  subset="both",
  seed=42,
  image_size=(config.img_size, config.img_size),
  batch_size=config.batch_size)

In [ ]:

config.class_names = train_loader.class_names
config.num_classes = len(config.class_names)
print(config.class_names)

In [ ]:
def plot_images(dataset, class_names, num_images=25):
    # Get a batch of images and labels from the dataset
    image_batch, label_batch = next(iter(dataset))

    plt.figure(figsize=(10, 10))
    sns.set_style("whitegrid")
    for i in range(num_images):
        ax = plt.subplot(5, 5, i + 1)
        plt.imshow(image_batch[i].numpy().astype("uint8"))
        label = class_names[label_batch[i]]
        plt.title(label)
        plt.axis("off")
    plt.tight_layout()

plot_images(train_loader, class_names=train_loader.class_names)

In [ ]:
def plot_class_distribution(train_ds, test_ds, class_names):
    # Get the class distributions in the datasets
    train_class_distribution = Counter([class_names[label] for images, labels in train_ds for label in labels])
    test_class_distribution = Counter([class_names[label] for images, labels in test_ds for label in labels])

    # Sort the keys and values of the class distributions
    train_keys, train_values = zip(*sorted(train_class_distribution.items()))
    test_keys, test_values = zip(*sorted(test_class_distribution.items()))

    # Create a figure with two subplots
    fig, axs = plt.subplots(1, 2, figsize=(15, 5))

    # Plot the class distribution in the training dataset
    sns.barplot(x=list(train_keys), y=list(train_values), ax=axs[0], palette="Blues_d")
    axs[0].set_title('Class Distribution in Training Dataset')
    axs[0].tick_params(axis='x', rotation=45)

    # Plot the class distribution in the test dataset
    sns.barplot(x=list(test_keys), y=list(test_values), ax=axs[1], palette="Greens_d")
    axs[1].set_title('Class Distribution in Test Dataset')
    axs[1].tick_params(axis='x', rotation=45)

    plt.tight_layout()
    plt.show()

plot_class_distribution(train_loader, val_loader, config.class_names)

In [ ]:
rescaling = layers.Rescaling(1.0 / 255)
resize = layers.Resizing(config.img_size, config.img_size)
augmentation_layer = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical", seed=42),
    layers.RandomRotation(0.3, seed=42),
    layers.RandomZoom(0.3, seed=42),
    ])

In [ ]:
def create_model():
    tf.keras.backend.clear_session()
    model = tf.keras.Sequential([
    resize,
    tf.keras.Input(shape=(config.img_size, config.img_size, 3)),
    rescaling,
    augmentation_layer,
    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, padding='same', activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(),
    layers.Conv2D(128, 3, padding='same', activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(),
    layers.Conv2D(256, 3, padding='same', activation='relu'),
    layers.GlobalAveragePooling2D(),
    layers.Dense(512, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.25),
    layers.Dense(1,activation='sigmoid')
    ])
    return model


In [ ]:
model = create_model()
batch = next(iter(train_loader))
outputs = model(batch[0])
model.summary(expand_nested=True)

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
loss = tf.keras.losses.BinaryCrossentropy()
roc_auc = tf.keras.metrics.AUC(curve='ROC', name='roc_auc')
accuracy = tf.keras.metrics.BinaryAccuracy(name='accuracy')

In [ ]:
# define callbacks
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.1,
    patience=3,
    verbose=1,
    mode="auto",
    min_delta=0.0001,
    cooldown=0,
    min_lr=0,
)
checkpoint_filepath = "/content/model_checkpoints/models_resnet_50/model_{epoch:02d}_val_roc_auc_{val_roc_auc:.4f}_val_loss_{val_loss:.4f}_val_accuracy_{val_accuracy:.4f}.keras"
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor="val_loss",
    verbose=0,
    save_best_only=True,
    save_weights_only=True,
    mode="auto",
    save_freq="epoch",
)

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [ ]:
train_ds = train_loader.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_loader.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
model.compile(optimizer=optimizer, loss=loss, metrics=[roc_auc, accuracy])

In [ ]:
# If you don't have these installed
!pip install pydot
!apt-get install graphviz -y
from tensorflow.keras.utils import plot_model

# Create a diagram of the model architecture

plot_model(model, to_file='SMVNet_Model.png', show_shapes=True, show_layer_names=True)
from google.colab import files
files.download('SMVNet_Model.png')

In [ ]:
from google.colab import userdata
WANDB_API_KEY = userdata.get('wandb_api')

In [ ]:
wandb.login(key=WANDB_API_KEY)

In [ ]:
run = wandb.init(project="cortex", entity="shivanikotian18", job_type="train", group='CNN',tags=['CNN_Shivani'], config=config)
assert run is wandb.run, "Something went wrong"
#train and validate the model
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=config.epochs,
    callbacks=[checkpoint, WandbCallback(save_weights_only=False),reduce_lr, early_stopping],
)
run.finish()